In [1]:
from __future__ import print_function

import numpy as np
from torch.utils import data
import argparse
import torch
import torch.utils.data as data_utils
import torch.optim as optim
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
#2021, 3031,600,2455
seed = 9427
torch.manual_seed(seed)
torch.cuda.manual_seed(0)
np.random.seed(seed)

# torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)
    
    
class Dataset(data.Dataset):
    def __init__(self, X1,Y):
        self.X1 = X1
        self.Y = Y
    def __len__(self):        
        return len(self.X1)
    
    def __getitem__(self, index):
        x = self.X1[index]
        y = self.Y[index]
        return x,y

#from model import Attention, GatedAttention

## Negetive pictures

In [2]:
import os
def readFeatures(path):
    features=np.array([])
    start=0
    for file in os.listdir(path):
        d = os.path.join(path, file)
        datafile=d+"/features.npy"
        try:
            temp=np.load(datafile)
#             print(datafile)
#             print(temp.shape)
            if(start==0):
                features=temp
                if(d[-10]=='S'):
                    label=np.ones(len(temp))
                else:
                    label=np.zeros(len(temp))
                start=1
            else:
                features=np.vstack((features,temp))
                if(d[-10]=='S'):
                    label=np.append(label,np.ones(len(temp)))
                else:
                    label=np.append(label,np.zeros(len(temp)))
        except IOError:
            print('file not in Scripted')

            
    return features,label

def load_data(path,mode):
    if mode == "test":
        xTest,yTest=readFeatures(path+"/"+mode)
        xTest = xTest.reshape(len(xTest),1,19,8)
        return Dataset(xTest,yTest)
        
    elif mode == "train":
        xTrain,yTrain=readFeatures(path+"/"+mode)
        xTrain = xTrain.reshape(len(xTrain),1,19,8)
        return Dataset(xTrain,yTrain)
        
    elif mode == "val":
        xVal,yVal=readFeatures(path+"/"+mode)
        xVal = xVal.reshape(len(xVal),1,19,8)
        return Dataset(xVal,yVal)
    else:
        print("Mode not defined")
        return

traindataset = load_data("../Participant_wise_physio_change",'train')
testdataset = load_data("../Participant_wise_physio_change",'test')
valdataset = load_data("../Participant_wise_physio_change",'val')

In [3]:
# xTrain=np.load("../extracted_features/Change/input_features_train_grp3.npy")
# labels_stress_train=np.load("../extracted_features/Change/labels_stress_train_grp3.npy")
# yTrain=np.load("../extracted_features/Change/labels_train_grp3.npy")

# xTest=np.load("../extracted_features/Change/input_features_test_grp3.npy")
# labels_stress_test=np.load("../extracted_features/Change/labels_stress_test_grp3.npy")
# yTest=np.load("../extracted_features/Change/labels_test_grp3.npy")


# xVal=np.load("../extracted_features/Change/input_features_val_grp3.npy")
# labels_stress_val=np.load("../extracted_features/Change/labels_stress_val_grp3.npy")
# yVal=np.load("../extracted_features/Change/labels_val_grp3.npy")


In [4]:
# xTrain.shape

In [5]:
# xVal.shape

In [6]:
# xTest.shape

In [7]:
# xTrain = xTrain.reshape(len(xTrain),1,19,8)
# xTest = xTest.reshape(len(xTest),1,19,8)
# xVal = xVal.reshape(len(xVal),1,19,8)

In [8]:
# print(yTest)

In [9]:
# print(xTrain.shape,yTrain.shape)
# print(xTest.shape,yTest.shape)
# print(xVal.shape,yVal.shape)

In [10]:
# traindataset = Dataset(xTrain,yTrain)
# testdataset = Dataset(xTest,yTest)
# valdataset = Dataset(xVal,yVal)

In [11]:
class Stammering(data_utils.Dataset):
    def __init__(self, target_number=1, mean_bag_length=5, var_bag_length=2, num_bag=150, seed=2021, train="train"):
        self.target_number = target_number
        self.mean_bag_length = mean_bag_length
        self.var_bag_length = var_bag_length
        self.num_bag = num_bag
        self.train = train
        self.r = np.random.RandomState(seed)



        if self.train=="train":
            self.train_bags_list, self.train_labels_list = self._create_bags()
        elif self.train=="val":
            self.val_bags_list, self.val_labels_list = self._create_bags()
        else:
            self.test_bags_list, self.test_labels_list = self._create_bags()

    def _create_bags(self):
        if self.train=="train":
            print("train")
            loader = data_utils.DataLoader(traindataset,
                                           batch_size=1,
                                           shuffle=True)
        elif self.train=="val":
            print("val")
            loader = data_utils.DataLoader(valdataset,
                                           batch_size=1,
                                           shuffle=True)
        else:
            loader = data_utils.DataLoader(testdataset,
                                           batch_size=1,
                                           shuffle=True)
            
        bags_list = []
        labels_list = []
        for (batch_data, batch_labels) in loader:
            #print(batch_data.shape)
            bags_list.append(batch_data.reshape(19,1,8))
            temp = torch.as_tensor(np.array([batch_labels for x in range(19)]))
            labels_list.append(temp)
            
               
        #print(bags_list)
        #print(labels_list)
        return bags_list, labels_list

    def __len__(self):
        if self.train=="train":
            return len(self.train_labels_list)
        elif self.train=="val":
            return len(self.val_labels_list)
        else:
            return len(self.test_labels_list)

    def __getitem__(self, index):
        if self.train=="train":
            bag = self.train_bags_list[index]
            label = [max(self.train_labels_list[index]), self.train_labels_list[index]]
        elif self.train=="val":
            bag = self.val_bags_list[index]
            label = [max(self.val_labels_list[index]), self.val_labels_list[index]]
        else:
            bag = self.test_bags_list[index]
            label = [max(self.test_labels_list[index]), self.test_labels_list[index]]

        return bag, label

In [12]:
train_loader = data_utils.DataLoader(Stammering(train="train"),batch_size=1,worker_init_fn=seed_worker,shuffle=True)
test_loader = data_utils.DataLoader(Stammering(train="test"),batch_size=1,worker_init_fn=seed_worker,shuffle=True)
val_loader = data_utils.DataLoader(Stammering(train="val"),batch_size=1,worker_init_fn=seed_worker,shuffle=True)

train
val


In [13]:
class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        self.L = 256
        self.D = 64
        self.K = 1

        self.feature_extractor_part1 = nn.Sequential(
            nn.Conv1d(1, 128, kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv1d(128, 64, kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Flatten()
        )

        self.feature_extractor_part2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(256),
            nn.Linear(256,self.L),
            nn.ReLU()

        )

        self.attention = nn.Sequential(
            nn.Linear(self.L, self.D),
            nn.Tanh(),
            nn.Linear(self.D, self.K)
        )

        self.classifier = nn.Sequential(

            nn.Linear(self.L*self.K, 200),
            nn.ReLU(),
            nn.Linear(200, 150),
            nn.ReLU(),
            nn.Linear(150, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )


    def forward(self, x):
        x=x.squeeze(0)

        H = self.feature_extractor_part1(x)
        
        #H = H.view(-1, 256*5*4)
        H = H.view(H.size(0), -1)
        #print(H.shape)
        H = self.feature_extractor_part2(H)  # NxL

        A = self.attention(H)  # NxK

        A = torch.transpose(A, 1, 0)  # KxN

        A = F.softmax(A, dim=1)  # softmax over N

        M = torch.mm(A, H)  # KxL

        Y_prob = self.classifier(M)
        Y_hat = torch.ge(Y_prob, 0.5).float()

        return Y_prob



    # AUXILIARY METHODS
    def calculate_classification_error(self, X, Y):
        Y = Y.float()
        Y_prob= self.forward(X)
        Y_hat = torch.ge(Y_prob, 0.5).float()
        error = 1. - Y_hat.eq(Y).cpu().float().mean().item()

        return error, Y_hat

    def calculate_objective(self, X, Y):
        Y = Y.float()
        Y_prob = self.forward(X)
        Y_prob = torch.clamp(Y_prob, min=1e-5, max=1. - 1e-5)
        neg_log_likelihood = -1. * (Y * torch.log(Y_prob) + (1. - Y) * torch.log(1. - Y_prob))  # negative log bernoulli

        return neg_log_likelihood

In [14]:
model = Attention()
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999))

In [15]:
# from torchsummary import summary
# summary(model.cuda(),(1,8))
# # torch.save(model, "modelarchitect_attn.pt")

In [16]:
model = Attention()
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999))
if torch.cuda.is_available():model.cuda()
best_model = "./saved_models/best_model_Attention-mil-scripted-change-grp3"
train_acc = []
modelname=[]
val_acc = []
best_acc = 0
for epoch in range(0, 45):
    model.train()
    train_loss = 0.
    train_error = 0.
    y =[]
    ypred = []
    for batch_idx, (data, label) in enumerate(train_loader):
        bag_label = label[0]
        data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)
        optimizer.zero_grad()
        loss = model.calculate_objective(data.float(), bag_label.float())
        train_loss += loss.data[0]
        error,y_pred = model.calculate_classification_error(data.float(), bag_label.float())
        train_error += error
        y_pred=y_pred.squeeze(1)
        ypred.extend(y_pred.tolist())
        y.extend(bag_label.tolist())
        loss.mean().backward()
        optimizer.step()
    
    trainacc = f1_score(y,ypred)
    train_loss /= len(train_loader)
    train_error /= len(train_loader)
    print("EPOCH ",epoch)
    print('Train : Loss: {:.4f}, Train error: {:.4f}, Train acc : {}'.format(train_loss.cpu().numpy()[0], 
                                                                                train_error,trainacc))
    y =[]
    ypred = []
    val_error=0.
    val_loss= 0.
    model.eval()
    for batch_idx, (data, label) in enumerate(val_loader):
        bag_label = label[0]
        data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)
        loss = model.calculate_objective(data.float(), bag_label.float())
        val_loss += loss.data[0]
        error, y_pred = model.calculate_classification_error(data.float(), bag_label.float())
        val_error += error
        y_pred=y_pred.squeeze(1)

        ypred.extend(y_pred.tolist())
        y.extend(bag_label.tolist())
    valacc = f1_score(y,ypred)
    val_loss /= len(val_loader)
    val_error /= len(val_loader)
    print('Val : Loss: {:.4f}, val error: {:.4f}, Val acc :{}'.format(val_loss.cpu().numpy()[0], val_error,valacc))
    if valacc>=best_acc:
        print("---------State saved---------")
        best_acc = valacc
        best_state=model.state_dict()
        torch.save(best_state, best_model+'_epoch_'+str(epoch)+".pth")
        modelname.append(best_model+'_epoch_'+str(epoch)+".pth")
    print('Best validation accuracy ',best_acc)
    test_loss = 0.
    correct = 0
    total = 0
    y =[]
    ypred = []
    ypred1=[]
    test_error = 0.
    for batch_idx, (data, label) in enumerate(test_loader):
        bag_label = label[0]
        instance_labels = label[1]
        data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)
        y_pred = model(data.float())
        error, predicted_label = model.calculate_classification_error(data.float(), bag_label.float())
        test_error += error
        predicted_label=predicted_label.squeeze(1)
        y_pred=y_pred.squeeze(1)
        ypred.extend(predicted_label.tolist())
        y.extend(bag_label.tolist())
    testacc = f1_score(y,ypred)
    test_error /= len(test_loader)
    print('test :test error: {:.4f}, Val acc :{}'.format(test_error,testacc))
        

EPOCH  0
Train : Loss: 0.6898, Train error: 0.4612, Train acc : 0.0
Val : Loss: 0.6919, val error: 0.4746, Val acc :0.06666666666666667
---------State saved---------
Best validation accuracy  0.06666666666666667
test :test error: 0.4576, Val acc :0.18181818181818182
EPOCH  1
Train : Loss: 0.6779, Train error: 0.4360, Train acc : 0.3119266055045871
Val : Loss: 0.7675, val error: 0.6102, Val acc :0.0
Best validation accuracy  0.06666666666666667
test :test error: 0.4576, Val acc :0.2058823529411765
EPOCH  2
Train : Loss: 0.6705, Train error: 0.3973, Train acc : 0.46475195822454307
Val : Loss: 0.7677, val error: 0.5932, Val acc :0.2708333333333333
---------State saved---------
Best validation accuracy  0.2708333333333333
test :test error: 0.4068, Val acc :0.5555555555555556
EPOCH  3
Train : Loss: 0.6559, Train error: 0.3682, Train acc : 0.5497630331753555
Val : Loss: 0.7916, val error: 0.5339, Val acc :0.5190839694656488
---------State saved---------
Best validation accuracy  0.5190839694

Val : Loss: 3.4377, val error: 0.5000, Val acc :0.5874125874125875
Best validation accuracy  0.625
test :test error: 0.3305, Val acc :0.7022900763358779
EPOCH  35
Train : Loss: 0.4500, Train error: 0.2267, Train acc : 0.7370786516853932
Val : Loss: 3.0005, val error: 0.5932, Val acc :0.43548387096774194
Best validation accuracy  0.625
test :test error: 0.2627, Val acc :0.6804123711340206
EPOCH  36
Train : Loss: 0.4543, Train error: 0.2190, Train acc : 0.7378190255220417
Val : Loss: 3.1868, val error: 0.5424, Val acc :0.5294117647058824
Best validation accuracy  0.625
test :test error: 0.3051, Val acc :0.7
EPOCH  37
Train : Loss: 0.4365, Train error: 0.2074, Train acc : 0.7482352941176471
Val : Loss: 3.6075, val error: 0.4661, Val acc :0.5985401459854014
Best validation accuracy  0.625
test :test error: 0.3390, Val acc :0.696969696969697
EPOCH  38
Train : Loss: 0.4493, Train error: 0.1957, Train acc : 0.7688787185354692
Val : Loss: 1.6788, val error: 0.4492, Val acc :0.41758241758241754

In [17]:
device = torch.device("cuda")
model=Attention()
# best_state=torch.load("../best-models/Attention_MIL_scripted_change_F1_79.pth")
model.load_state_dict(best_state)
model.to(device)
model.eval()
test_loss = 0.
correct = 0
total = 0
y =[]
ypred = []
ypred1=[]
test_error = 0.
for batch_idx, (data, label) in enumerate(test_loader):
    bag_label = label[0]
    instance_labels = label[1]
    data, bag_label = data.cuda(), bag_label.cuda()
    data, bag_label = Variable(data), Variable(bag_label)
    y_pred = model(data.float())
    error, predicted_label = model.calculate_classification_error(data.float(), bag_label.float())
    test_error += error
    predicted_label=predicted_label.squeeze(1)
    y_pred=y_pred.squeeze(1)
    ypred.extend(predicted_label.tolist())
    ypred1.extend(y_pred.tolist())
    y.extend(bag_label.tolist())
    
acc=accuracy_score(y,ypred)
# print(y,ypred)
tn, fp, fn, tp = confusion_matrix(y,ypred).ravel()
f1score=f1_score(y,ypred)
precision=precision_score(y,ypred)
recall=recall_score(y,ypred)
roc=roc_auc_score(y,ypred)
specificity=tn/(tn+fp)
print(acc,f1score,precision,recall,roc,specificity)


0.652542372881356 0.7092198581560283 0.6024096385542169 0.8620689655172413 0.6560344827586206 0.45


## Window1 and Window-2

In [18]:
testdataset = load_data("../Participant_wise_physio_change1",'test')
test_loader = data_utils.DataLoader(Stammering(train="test"),worker_init_fn=seed_worker,batch_size=1,shuffle=True)

device = torch.device("cuda")
model=Attention()
# best_state=torch.load("../best-models/Attention_MIL_scripted_change_F1_79.pth")
best_state=torch.load(modelname[-1])
model.load_state_dict(best_state)
model.to(device)
model.eval()
test_loss = 0.
correct = 0
total = 0
y =[]
ypred = []
test_error = 0.
for batch_idx, (data, label) in enumerate(test_loader):
    bag_label = label[0]
    instance_labels = label[1]
    data, bag_label = data.cuda(), bag_label.cuda()
    data, bag_label = Variable(data), Variable(bag_label)
    loss = model.calculate_objective(data.float(), bag_label.float())
    test_loss += loss.data[0]
    error, predicted_label = model.calculate_classification_error(data.float(), bag_label.float())
    test_error += error
    predicted_label=predicted_label.squeeze(1)
    ypred.extend(predicted_label.tolist())
    y.extend(bag_label.tolist())
    
acc=accuracy_score(y,ypred)
# print(y,ypred)
tn, fp, fn, tp = confusion_matrix(y,ypred).ravel()
f1score=f1_score(y,ypred)
precision=precision_score(y,ypred)
recall=recall_score(y,ypred)
roc=roc_auc_score(y,ypred)
specificity=tn/(tn+fp)
print(acc,f1score,precision,recall,roc,specificity)



0.6440677966101694 0.7272727272727274 0.5833333333333334 0.9655172413793104 0.6494252873563219 0.3333333333333333


In [19]:
testdataset = load_data("../Participant_wise_physio_change2",'test')
test_loader = data_utils.DataLoader(Stammering(train="test"),worker_init_fn=seed_worker,batch_size=1,shuffle=True)

device = torch.device("cuda")
model=Attention()
# best_state=torch.load("../best-models/Attention_MIL_scripted_change_F1_79.pth")
best_state=torch.load(modelname[-1])
model.load_state_dict(best_state)
model.to(device)
model.eval()
test_loss = 0.
correct = 0
total = 0
y =[]
ypred = []
test_error = 0.
for batch_idx, (data, label) in enumerate(test_loader):
    bag_label = label[0]
    instance_labels = label[1]
    data, bag_label = data.cuda(), bag_label.cuda()
    data, bag_label = Variable(data), Variable(bag_label)
    loss = model.calculate_objective(data.float(), bag_label.float())
    test_loss += loss.data[0]
    error, predicted_label = model.calculate_classification_error(data.float(), bag_label.float())
    test_error += error
    predicted_label=predicted_label.squeeze(1)
    ypred.extend(predicted_label.tolist())
    y.extend(bag_label.tolist())
    
acc=accuracy_score(y,ypred)
# print(y,ypred)
tn, fp, fn, tp = confusion_matrix(y,ypred).ravel()
f1score=f1_score(y,ypred)
precision=precision_score(y,ypred)
recall=recall_score(y,ypred)
roc=roc_auc_score(y,ypred)
specificity=tn/(tn+fp)
print(acc,f1score,precision,recall,roc,specificity)


0.7288135593220338 0.7837837837837839 0.6444444444444445 1.0 0.7333333333333334 0.4666666666666667


In [20]:
# for i in range(len(modelname)):
#     print(modelname[i])
#     device = torch.device("cuda")
#     model=Attention()
#     # best_state=torch.load("../best-models/Attention_MIL_scripted_change_F1_79.pth")
#     best_state=torch.load(modelname[i])
#     model.load_state_dict(best_state)
#     model.to(device)
#     model.eval()
#     test_loss = 0.
#     correct = 0
#     total = 0
#     y =[]
#     ypred = []
#     test_error = 0.
#     for batch_idx, (data, label) in enumerate(test_loader):
#         bag_label = label[0]
#         instance_labels = label[1]
#         data, bag_label = data.cuda(), bag_label.cuda()
#         data, bag_label = Variable(data), Variable(bag_label)
#         loss = model.calculate_objective(data.float(), bag_label.float())
#         test_loss += loss.data[0]
#         error, predicted_label = model.calculate_classification_error(data.float(), bag_label.float())
#         test_error += error
#         predicted_label=predicted_label.squeeze(1)
#         ypred.extend(predicted_label.tolist())
#         y.extend(bag_label.tolist())

#     acc=accuracy_score(y,ypred)
#     # print(y,ypred)
#     tn, fp, fn, tp = confusion_matrix(y,ypred).ravel()
#     f1score=f1_score(y,ypred)
#     precision=precision_score(y,ypred)
#     recall=recall_score(y,ypred)
#     roc=roc_auc_score(y,ypred)
#     specificity=tn/(tn+fp)
#     print(acc,f1score,precision,recall,roc,specificity)

./saved_models/best_model_Attention-mil-scripted-change-grp3_epoch_0.pth
0.576271186440678 0.2424242424242424 1.0 0.13793103448275862 0.5689655172413793 1.0
./saved_models/best_model_Attention-mil-scripted-change-grp3_epoch_2.pth
0.6271186440677966 0.5925925925925927 0.64 0.5517241379310345 0.6258620689655172 0.7
./saved_models/best_model_Attention-mil-scripted-change-grp3_epoch_3.pth
0.6949152542372882 0.6538461538461539 0.7391304347826086 0.5862068965517241 0.6931034482758621 0.8
./saved_models/best_model_Attention-mil-scripted-change-grp3_epoch_4.pth
0.6610169491525424 0.6666666666666667 0.6451612903225806 0.6896551724137931 0.6614942528735632 0.6333333333333333
./saved_models/best_model_Attention-mil-scripted-change-grp3_epoch_5.pth
0.711864406779661 0.7605633802816901 0.6428571428571429 0.9310344827586207 0.7155172413793103 0.5
./saved_models/best_model_Attention-mil-scripted-change-grp3_epoch_8.pth
0.7288135593220338 0.7837837837837839 0.6444444444444445 1.0 0.7333333333333334 0.